In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import pymongo
import itertools
from concurrent.futures import ProcessPoolExecutor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
from sklearn.externals import joblib
from common.log import *
from common.config import Config

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']

In [2]:
symbol = 'TSLA'

stock_df_dict = {}

for symbol in NASDAQ:
    stock_data_file = '../database/market/%s.csv' % symbol
    stock_df = pd.read_csv(stock_data_file)

    # 筛选字段
    stock_df = stock_df.loc[:, ['date', 'open', 'close', 'volume']]

    # 去掉Nasdaq行情首行的当天行情
    if symbol in NASDAQ:
        stock_df = stock_df.drop([0])

    # 处理Coinmarketcap的早期缺失数据
    if symbol in CRYPTOCURRENCY:
        stock_df = stock_df.replace(to_replace='-', value=pd.np.nan)

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期，和news_df['publish_time']一致
    stock_df['date'] = stock_df['date'].apply(lambda x: pd.Period(x, freq='D'))

    # 转换字段格式
    stock_df = stock_df.astype(dtype={'volume': 'float64'})

    # 用日期作索引，日期升序排序
    stock_df.set_index(['date'], inplace=True)
    stock_df = stock_df.sort_index(axis=0, ascending=True)

    stock_df['pct_change'] = stock_df.close.pct_change(1)
    
    stock_df = stock_df['2017-01-01':]
    
    stock_df_dict[symbol] = stock_df

# stock_df
# df = stock_df.copy()

In [3]:
PROPERTY = 100000
hold = []

# print(stock_df_dict['TSLA'].index)
# print(pd.period_range(start='2017-01-01', end='2018-01-01', freq='D'))

yesterday = None

for today in pd.period_range(start='2017-01-01', end='2018-01-01', freq='D'):
#     print(date)
    if today not in stock_df_dict['TSLA'].index:
        continue
#     print(stock_df_dict['TSLA'].loc[date, :])

    for order in hold:
        if stock_df_dict[order[1]].loc[today, 'close'] > order[4]:
            PROPERTY += stock_df_dict[order[1]].loc[today, 'close'] * order[2]
            hold.remove(order)
            print('%s SELL %s * %d @ %f' % (today, order[1], order[2], stock_df_dict[order[1]].loc[today, 'close']))
    
    for symbol in NASDAQ:
#     for symbol in ['TSLA', 'NVDA', 'BIDU', 'FB']:
        if stock_df_dict[symbol].loc[today, 'pct_change'] < -0.05:
            buy = int(PROPERTY / 2 / stock_df_dict[symbol].loc[today, 'close'])
            PROPERTY -= stock_df_dict[symbol].loc[today, 'close'] * buy
            hold.append([today, symbol, buy, stock_df_dict[symbol].loc[today, 'close'], stock_df_dict[symbol].loc[yesterday, 'close']])
            print('%s BUY  %s * %d @ %f EXPECT %f' % (today, symbol, buy, stock_df_dict[symbol].loc[today, 'close'], stock_df_dict[symbol].loc[yesterday, 'close']))
    
    yesterday = today
            
print(PROPERTY)
print(hold)

PROPERTY = PROPERTY + sum([stock_df_dict[order[1]].loc[yesterday, 'close'] * order[2] for order in hold])

print(PROPERTY)

s = 0
c = 0
for symbol in NASDAQ:
# for symbol in ['TSLA', 'NVDA', 'BIDU', 'FB']:
    s_p = stock_df_dict[symbol].loc['2017-01-03', 'close']
    e_p = stock_df_dict[symbol].loc['2018-01-03', 'close']
    print(symbol, s_p, e_p, e_p / s_p)
    s += e_p / s_p
    c += 1

print(s / c)
    
# for symbol in NASDAQ:

# stock_df['HOLD'] = 0
# stock_df['PROPERTY'] = 100000

# for d in stock_df.index:
# #     print(stock_df.loc[d])
#     today = stock_df.loc[d]
    
# #     stock_df.loc[d, 'PROPERTY'] = stock_df.loc[d-1, 'PROPERTY']
# #     stock_df.loc[d, 'HOLD'] = stock_df.loc[d-1, 'HOLD']
    
#     for t, s in list(hold.items()):
#         if today.close > s[0]:
#             PROPERTY += today.close * s[1]
# #             print('SELL', d, s)
#             hold.pop(t)
# #             stock_df.loc[d, 'HOLD'] -= s[1]
    
#     if today['pct_change'] < -0.05:
#         if PROPERTY >= 10000:
#             buy = int(PROPERTY / 2 / today['close'])
#             PROPERTY -= buy * today['close']
#             hold[d] = [today['close'], buy]
# #             print('BUY', d, hold[d])
# #             stock_df.loc[d, 'HOLD'] += buy
    
#     stock_df.loc[d, 'PROPERTY'] = PROPERTY + sum([s[1] for t, s in hold.items()]) * today.close
#     stock_df.loc[d, 'HOLD'] = sum([s[1] for t, s in hold.items()])

# stock_df
# print(PROPERTY)
# print(hold)

2017-01-23 BUY  XNET * 12987 @ 3.850000 EXPECT 4.150000
2017-02-23 BUY  TSLA * 97 @ 255.990000 EXPECT 273.510000
2017-02-23 BUY  NVDA * 125 @ 100.490000 EXPECT 110.760000
2017-02-24 BUY  BIDU * 35 @ 175.170000 EXPECT 184.640000
2017-03-28 SELL TSLA * 97 @ 277.450000
2017-04-04 BUY  NVDA * 165 @ 100.780000 EXPECT 108.380000
2017-04-25 SELL BIDU * 35 @ 186.990000
2017-05-04 BUY  TSLA * 39 @ 295.460000 EXPECT 311.020000
2017-05-09 SELL TSLA * 39 @ 321.260000
2017-05-10 SELL NVDA * 125 @ 121.290000
2017-05-11 SELL NVDA * 165 @ 126.500000
2017-05-17 BUY  NVDA * 236 @ 127.720000 EXPECT 136.810000
2017-05-22 SELL NVDA * 236 @ 138.900000
2017-06-09 BUY  NVDA * 210 @ 149.600000 EXPECT 159.940000
2017-07-05 BUY  TSLA * 48 @ 327.090000 EXPECT 352.620000
2017-07-06 BUY  TSLA * 25 @ 308.830000 EXPECT 327.090000
2017-07-11 SELL TSLA * 25 @ 327.220000
2017-07-12 SELL NVDA * 210 @ 162.510000
2017-08-04 SELL TSLA * 48 @ 356.910000
2017-08-09 BUY  XNET * 9439 @ 3.580000 EXPECT 3.790000
2017-08-10 BUY  N